In [5]:
import json
import pandas as pd

#读取json
with open('./config/config.json','r+') as file:
    content=file.read()
config=json.loads(content)

In [6]:
def judge(fea_data,config,way):
    rate=0
    config_dict=config[way]
    fea_list=list(config_dict.keys())
    for fea in fea_list:
        value=fea_data[fea]
        bins=config_dict[fea]["bins"]
        rates = config_dict[fea]["rate"]

        if value<min(bins):
            rate=rate+rates[0]
        elif value>max(bins):
            rate=rate+rates[-1]
        else:
            for b in range(len(bins)-1):
                if value>bins[:-1][b] and value<=bins[1:][b]:
                    rate=rate+rates[b]
    if rate>len(fea_list)*0.5:
        return 1
    else:
        return 0


##### 开始运行

In [7]:
tit='N_all.csv'
# tit='P_all.csv'
# tit='P_ice.csv'
# tit='P_her.csv'
# tit='P_mix.csv'
# tit='P_ice1.csv'

In [8]:
path = 'E:\\desktop\\data\\app\\fea\\'
#读取数据
data = pd.read_csv(path+tit,encoding='gbk')
data = data.set_index('name')

result={'name':[],'all':[],'ice':[],'her':[],'mix':[]}
name_list=list(data.index)
for i in range(len(data)):
    row=data.iloc[i]
    result['name'].append(name_list[i])
    fea_data=dict(row)
    # print(fea_data)
    for way in ("all","ice","her","mix"):
        result[way].append(judge(fea_data,config,way))

result=pd.DataFrame(result)
result=result.set_index('name')

youla={"(ice or her or mix) and all":[],"(ice or her) and all":[],"四选二":[]}
for i in range(len(result)):
    row=list(result.iloc[i])
    if sum(row[:3])>=1 and row[3]==1:
        youla["(ice or her or mix) and all"].append(1)
    else:
        youla["(ice or her or mix) and all"].append(0)

    if sum(row[:2])>=1 and row[3]==1:
        youla["(ice or her) and all"].append(1)
    else:
        youla["(ice or her) and all"].append(0)

    if sum(row)>=2:
        youla["四选二"].append(1)
    else:
        youla["四选二"].append(0)
youla=pd.DataFrame(youla,index=result.index)
result=pd.concat([result,youla],axis=1)

psum=len(result)
for key in result.columns:
    p=sum(result[key])
    print(key,p,'/',psum,p/psum)


all 35 / 523 0.06692160611854685
ice 23 / 523 0.04397705544933078
her 35 / 523 0.06692160611854685
mix 29 / 523 0.055449330783938815
(ice or her or mix) and all 16 / 523 0.030592734225621414
(ice or her) and all 14 / 523 0.02676864244741874
四选二 32 / 523 0.06118546845124283
